### Data Fetching

In [1]:
!pip install yfinance

In [2]:
import yfinance as yf
import pandas as pd
import requests
import bs4 as bs

In [5]:
resp = requests.get("http://en.wikipedia.org/wiki/List_of_S%26P_500_companies")

'''
soup = bs.BeautifulSoup(resp.text, "lxml")
table = soup.find("table", { "class": "wikitable sortable" })
tickers = []
for row in table.findAll("tr")[1:]:
  ticker = row.findAll("td")[0].text
  tickers.append(ticker.strip())'''

'\nsoup = bs.BeautifulSoup(resp.text, "lxml")\ntable = soup.find("table", { "class": "wikitable sortable" })\ntickers = []\nfor row in table.findAll("tr")[1:]:\n  ticker = row.findAll("td")[0].text\n  tickers.append(ticker.strip())'

In [6]:
data = download_sp500_data()

NameError: name 'download_sp500_data' is not defined

In [7]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Use requests with a proper User-Agent header to avoid being blocked
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Get the page content
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
response = requests.get(url, headers=headers)

# Parse the HTML
soup = BeautifulSoup(response.text, 'html.parser')

# Find the first table on the page (which should be the S&P 500 companies table)
table_html = str(soup.find('table', {'class': 'wikitable'}))

# Use pd.read_html on the extracted table HTML
table = pd.read_html(table_html)[0]



C:\Users\mohdz\AppData\Local\Temp\ipykernel_28384\4274665274.py:21: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(table_html)[0]


In [8]:
table['Symbol'].to_csv('sp500_symbols.csv', index=False)

### Reading tickers through yfinance and finding returns

In [9]:

def download_sp500_data():
    # Step 1: Get the S&P 500 ticker list from Wikipedia
    print("Fetching S&P 500 ticker list...")
    # Clean tickers (Yahoo uses '-' instead of '.' for symbols like BRK.B)
    tickers = pd.read_csv('sp500_symbols.csv')
    tickers = tickers['Symbol'].to_list()
    print(f"Found {len(tickers)} tickers. Starting download...")

    # Step 2: Download the data
    # We use yf.download because it's optimized for bulk requests
    # period="1y" gives you the last year of data; change to "max" for everything
    data = yf.download(tickers[:10], period="1y", threads=True)

    
    # Save the consolidated file
    #data.to_csv("sp10_historical_data.csv")
    
    return data

data = download_sp500_data()

Fetching S&P 500 ticker list...
Found 503 tickers. Starting download...


[*********************100%***********************]  10 of 10 completed


In [10]:
data.head()

Price            Close                                                  \
Ticker               A        ABBV         ABT         ACN        ADBE   
Date                                                                     
2025-02-21  134.288986  195.521011  132.435028  360.658875  444.320007   
2025-02-24  134.199677  197.456085  132.464478  360.312317  444.420013   
2025-02-25  133.108475  197.514130  133.318451  359.361816  443.410004   
2025-02-26  133.396179  196.420807  133.455872  354.807281  441.500000   
2025-02-27  126.065186  198.365570  133.367538  353.341919  437.190002   

Price                                                                 ...  \
Ticker            AES         AFL         AMD        AOS         MMM  ...   
Date                                                                  ...   
2025-02-21  10.016912   99.992035  110.839996  64.657867  142.356628  ...   
2025-02-24  10.139878  101.881065  108.110001  64.902855  142.847580  ...   
2025-02-25  10.272300  104.465050  103.959999  65.647621  143.888382  ...   
2025-02-26  10.329055  103.564568  104.739998  64.834259  144.762299  ...   
2025-02-27   9.818277  105.150185   99.510002  64.618668  147.796387  ...   

Price        Volume                                                         \
Ticker            A     ABBV      ABT      ACN     ADBE       AES      AFL   
Date                                                                         
2025-02-21  1112200  5889400  7160100  3751700  3376000  19723000  3539600   
2025-02-24  2037500  6305000  5947200  2755800  2807900  10345900  2475400   
2025-02-25  2607000  7746800  6385700  1947600  2612700  12792000  2147200   
2025-02-26  1539900  4766700  5388000  2267500  1947600  12192600  1586200   
2025-02-27  3469100  5696900  4890700  2735400  2340200  16285600  1831100   

Price                                   
Ticker           AMD      AOS      MMM  
Date                                    
2025-02-21  41353700  1097800  4278500  
2025-02-24  29072200   926800  3447500  
2025-02-25  39169200  1316600  4358800  
2025-02-26  29178200  1091100  5783400  
2025-02-27  47263700   795100  6576000  

[5 rows x 50 columns]

In [11]:
opens = data.xs('Open', level=0, axis=1)
closes = data.xs('Close', level=0, axis=1)
rets = (opens - closes) / opens
closes

Ticker,A,ABBV,ABT,ACN,ADBE,AES,AFL,AMD,AOS,MMM
Date,,,,,,,,,,
2025-02-21,134.288986,195.521011,132.435028,360.658875,444.320007,10.016912,99.992035,110.839996,64.657867,142.356628
2025-02-24,134.199677,197.456085,132.464478,360.312317,444.420013,10.139878,101.881065,108.110001,64.902855,142.847580
2025-02-25,133.108475,197.514130,133.318451,359.361816,443.410004,10.272300,104.465050,103.959999,65.647621,143.888382
2025-02-26,133.396179,196.420807,133.455872,354.807281,441.500000,10.329055,103.564568,104.739998,64.834259,144.762299
2025-02-27,126.065186,198.365570,133.367538,353.341919,437.190002,9.818277,105.150185,99.510002,64.618668,147.796387
...,...,...,...,...,...,...,...,...,...,...
2026-02-13,125.809998,231.500000,112.680000,224.229996,263.970001,16.280001,114.302704,207.320007,80.459999,171.820007
2026-02-17,123.870003,232.860001,112.260002,219.889999,260.450012,16.389999,114.809998,203.080002,79.470001,167.619995
2026-02-18,126.940002,228.720001,113.290001,223.610001,263.170013,16.290001,113.620003,200.119995,78.959999,164.169998


### Momentum with last year daily data 

In [12]:
lookback = 126
hold_out = 21

delayed_price = closes.shift(hold_out)
starting_price = closes.shift(hold_out + lookback)

mom_score = (delayed_price/starting_price) - 1
mom_score

Ticker,A,ABBV,ABT,ACN,ADBE,AES,AFL,AMD,AOS,MMM
Date,,,,,,,,,,
2025-02-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-02-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-02-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-02-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-02-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2026-02-13,0.291319,0.172681,-0.046942,0.031716,-0.158471,0.103711,0.097782,0.396802,0.056744,0.088533
2026-02-17,0.249400,0.140609,0.034834,0.017934,-0.170173,0.123727,0.092319,0.420859,0.027983,0.085445
2026-02-18,0.229674,0.149912,-0.006057,0.013348,-0.190465,0.089405,0.086928,0.476718,0.041605,0.104873


In [13]:
all_daily_ranks = mom_score.rank(axis=1, pct=True)
all_daily_ranks

Ticker,A,ABBV,ABT,ACN,ADBE,AES,AFL,AMD,AOS,MMM
Date,,,,,,,,,,
2025-02-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-02-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-02-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-02-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-02-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2026-02-13,0.9,0.8,0.2,0.3,0.1,0.7,0.6,1.0,0.4,0.5
2026-02-17,0.9,0.8,0.4,0.2,0.1,0.7,0.6,1.0,0.3,0.5
2026-02-18,0.9,0.8,0.2,0.3,0.1,0.6,0.5,1.0,0.4,0.7


In [14]:
current_top_5 = all_daily_ranks.iloc[-1].sort_values(ascending=False).head(3).index.to_list()
current_top_5

['AMD', 'A', 'ABBV']

In [57]:
def sp500_stockprice(period='1y', interval='1d'):
    '''
    Downloads S&P500 stock prices given a period and interval and returns the dataframe.
    '''

    # Reading ticker symbols and converting to list format
    tickers = pd.read_csv('sp500_symbols.csv')
    tickers = tickers['Symbol'].to_list()

    # Downloading data for each stock from yahoo finance.
    # Retrieves multi column dataframe with open,close,min,max,volume at first level and ticker labels at second level
    data = yf.download(tickers, start='2020-01-01' , interval=interval, threads=True)

    return data

def mom_returns(stocks_df, lookback, lag, num_of_stocks):

    closes = stocks_df.xs('Close', level=0, axis=1)

    lookback = lookback
    hold_out = lag
    
    delayed_price = closes.shift(hold_out)
    starting_price = closes.shift(hold_out + lookback)
    
    mom_score = (delayed_price/starting_price) - 1

    all_daily_ranks = mom_score.rank(axis=1, pct=True)

    long_pos = all_daily_ranks.iloc[-1].sort_values(ascending=False).head(num_of_stocks).index.to_list()
    short_pos = all_daily_ranks.iloc[-1].sort_values(ascending=False).tail(num_of_stocks).index.to_list()

    print('Go Long on ', long_pos)
    print('Go Short on ', short_pos)
    

In [58]:
sp500 = sp500_stockprice()
mom_returns(sp500, 126, 21, 10)

[**********            21%                       ]  108 of 503 completed$BF.B: possibly delisted; no price data found  (1d 2020-01-01 -> 2026-02-20)
[**********************87%*****************      ]  440 of 503 completed$BRK.B: possibly delisted; no timezone found
[*********************100%***********************]  503 of 503 completed

2 Failed downloads:
['BF.B']: possibly delisted; no price data found  (1d 2020-01-01 -> 2026-02-20)
['BRK.B']: possibly delisted; no timezone found


Go Long on  ['SNDK', 'WDC', 'MU', 'CIEN', 'TER', 'STX', 'LRCX', 'INTC', 'WBD', 'ALB']
Go Short on  ['SMCI', 'PAYC', 'GDDY', 'COIN', 'CHTR', 'TTD', 'FISV', 'BF.B', 'BRK.B', 'Q']


In [17]:
aapl = yf.download('AAPL')

[*********************100%***********************]  1 of 1 completed


In [25]:
!pip install pandas_datareader

In [30]:
import yfinance as yf
import pandas_datareader.data as web
import pandas as pd
import statsmodels.api as sm

# 1. Define parameters
ticker = 'AAPL'
start_date = '2020-01-01'
end_date = '2023-12-31'

# 2. Pull Stock Data
# We use .squeeze() to ensure we get a Series even if yfinance returns a DataFrame
df_stock = yf.download(ticker, start=start_date, end=end_date)['Close'].squeeze()

# 3. Process Stock Returns
# Convert to Monthly, calculate % change, and crucially: convert to PeriodIndex ('M')
stock_returns = df_stock.resample('ME').last().pct_change().dropna()
stock_returns.index = stock_returns.index.to_period('M') 
stock_returns.name = 'Returns'

# 4. Pull Fama-French 3 Factors
# These already come as PeriodIndex ('M') from pandas_datareader
ff_factors = web.DataReader('F-F_Research_Data_Factors', 'famafrench', start_date, end_date)[0]
ff_factors = ff_factors / 100

# 5. Merge
# Now that both indices are PeriodIndex('M'), they will align perfectly
data = pd.merge(stock_returns, ff_factors, left_index=True, right_index=True)

# 6. Calculate Excess Returns
data['Excess_Ret'] = data['Returns'] - data['RF']

# 7. Run the Regression (The "Fitting" part)
Y = data['Excess_Ret']
X = data[['Mkt-RF', 'SMB', 'HML']]
X = sm.add_constant(X)  # Adds the Intercept/Alpha

model = sm.OLS(Y, X).fit()

print("--- Regression Results ---")
print(model.summary())

[*********************100%***********************]  1 of 1 completed


--- Regression Results ---
                            OLS Regression Results                            
Dep. Variable:             Excess_Ret   R-squared:                       0.741
Model:                            OLS   Adj. R-squared:                  0.723
Method:                 Least Squares   F-statistic:                     41.00
Date:                Sat, 21 Feb 2026   Prob (F-statistic):           1.12e-12
Time:                        02:21:40   Log-Likelihood:                 78.428
No. Observations:                  47   AIC:                            -148.9
Df Residuals:                      43   BIC:                            -141.5
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0116    

C:\Users\mohdz\AppData\Local\Temp\ipykernel_28384\1424698550.py:23: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff_factors = web.DataReader('F-F_Research_Data_Factors', 'famafrench', start_date, end_date)[0]
C:\Users\mohdz\AppData\Local\Temp\ipykernel_28384\1424698550.py:23: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff_factors = web.DataReader('F-F_Research_Data_Factors', 'famafrench', start_date, end_date)[0]


In [38]:
data

,Returns,Mkt-RF,SMB,HML,RF,Excess_Ret
Date,,,,,,
2020-02,-0.114701,-0.0815,0.0108,-0.0382,0.0012,-0.115901
2020-03,-0.069762,-0.1337,-0.0469,-0.1383,0.0012,-0.070962
2020-04,0.155374,0.1360,0.0250,-0.0134,0.0000,0.155374
2020-05,0.085095,0.0557,0.0240,-0.0500,0.0001,0.084995
2020-06,0.147386,0.0245,0.0267,-0.0217,0.0001,0.147286
2020-07,0.165132,0.0583,-0.0235,-0.0144,0.0001,0.165032
2020-08,0.216569,0.0762,-0.0020,-0.0302,0.0001,0.216469
2020-09,-0.102526,-0.0364,0.0006,-0.0272,0.0001,-0.102626
2020-10,-0.060012,-0.0208,0.0434,0.0425,0.0001,-0.060112


In [50]:
def calculate_umd_vectorized(stocks_df, x_percent=0.3):
    # 1. Monthly Closes and Returns
    monthly_closes = stocks_df['Close'].resample('ME').last()
    monthly_returns = monthly_closes.pct_change() # Returns at time T
    
    # 2. Momentum Score (using your 12-1 logic)
    # Score at time T = Price(T-1) / Price (T-12)
    mom_score = (monthly_closes.shift(1) / monthly_closes.shift(6)) - 1
    
    # 3. Rank the scores across columns (axis=1)
    ranks = mom_score.rank(axis=1, pct=True)
    
    # 4. Identify Winners and Losers
    # We create boolean masks
    winners_mask = (ranks >= (1 - x_percent))
    losers_mask = (ranks <= x_percent)
    
    # 5. Apply masks to the returns and calculate the mean for each row
    # monthly_returns.where(condition) keeps the value if true, else NaN
    win_returns = monthly_returns.where(winners_mask).mean(axis=1)
    lose_returns = monthly_returns.where(losers_mask).mean(axis=1)
    
    # 6. Final Factor Calculation
    umd_series = win_returns - lose_returns
    umd_df = umd_series.to_frame(name='UMD').dropna()
    umd_df.index = umd_df.index.to_period('M')
    
    return umd_df

In [56]:
sp500.tail()

Price      Adj Close                                             ...   Volume  \
Ticker           APP BF.B BRK.B CEG COIN EXE GEHC GEV HOOD KVUE  ...       WY   
Date                                                             ...            
2020-12-24       NaN  NaN   NaN NaN  NaN NaN  NaN NaN  NaN  NaN  ...  1135400   
2020-12-28       NaN  NaN   NaN NaN  NaN NaN  NaN NaN  NaN  NaN  ...  2541700   
2020-12-29       NaN  NaN   NaN NaN  NaN NaN  NaN NaN  NaN  NaN  ...  1988100   
2020-12-30       NaN  NaN   NaN NaN  NaN NaN  NaN NaN  NaN  NaN  ...  2128900   
2020-12-31       NaN  NaN   NaN NaN  NaN NaN  NaN NaN  NaN  NaN  ...  2633400   

Price                                                                      \
Ticker         WYNN      XEL       XOM     XYL       XYZ      YUM     ZBH   
Date                                                                        
2020-12-24   850900   598400   8039000  249700   8411200   340400  292314   
2020-12-28  1912000  1799800  23877500  408600  11640000  1822900  704520   
2020-12-29  1262500  1761400  20287700  501900  15453100  1818300  757771   
2020-12-30  2087100  1296400  23807300  418200   9837000  1267900  440428   
2020-12-31  2103400  1743300  22786500  504200   6872400  1651700  514176   

Price                        
Ticker        ZBRA      ZTS  
Date                         
2020-12-24  138300   417400  
2020-12-28  251200  1522400  
2020-12-29  185500  1188400  
2020-12-30  166100  1009000  
2020-12-31  176500  1292600  

[5 rows x 2529 columns]

In [59]:
calculate_umd_vectorized(sp500)

C:\Users\mohdz\AppData\Local\Temp\ipykernel_28384\2673611225.py:4: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  monthly_returns = monthly_closes.pct_change() # Returns at time T


,UMD
Date,
2020-07,0.070578
2020-08,-0.009993
2020-09,-0.003116
2020-10,0.008554
2020-11,-0.053709
...,...
2025-10,0.065200
2025-11,-0.012804
2025-12,-0.021044


In [69]:
hi = pd.read_csv('ff_monthly.csv')
hi

,Unnamed: 0,Mkt-RF,SMB,HML,RF
0,192607,2.89,-2.55,-2.39,0.22
1,192608,2.64,-1.14,3.81,0.25
2,192609,0.38,-1.36,0.05,0.23
3,192610,-3.27,-0.14,0.82,0.32
4,192611,2.54,-0.11,-0.61,0.31
...,...,...,...,...,...
1189,202508,1.84,3.87,4.42,0.38
1190,202509,3.39,-1.85,-1.05,0.33
1191,202510,1.96,-0.55,-3.09,0.37
1192,202511,-0.13,0.38,3.76,0.30
